In [14]:
% for loop iterating through participants in preprocessed folder
datapath = "/home/zachkaras/fmri/preprocessed/";
files = dir(datapath); 

for i=3:numel(files); 
    fnames{i-2}=files(i).name; 
end

In [18]:
addpath(genpath("/home/zachkaras/fmri/analysis"))

In [ ]:
for i=1:numel(fnames)
    tic
    name = regexp(fnames{i}, '.nii.gz', 'split');
    name = name{1};
    
    fprintf("regressing out nuissances for %s\n", name)

    brain_path = sprintf("/home/zachkaras/fmri/preprocessed/%s.nii.gz", name);
    brain_data = niftiread(brain_path);

    % find motion parameters file
    motion_filepath = sprintf("/home/zachkaras/fmri/three_studies_raw/out_%s/st_mc.nii.par", name);

    % design matrix and data
    X = make_design_matrix(motion_filepath);
    Y = reshape(brain_data, [(91*109*91),600])';

    % fitting parameters to brain signal
    b = X\Y;
    Yhat = X*b;
    YC = Y-Yhat;
    YC = reshape(YC', [91,109,91,600]);
    
    outfile = sprintf("/home/zachkaras/fmri/motion_corrected/%s_mc", name);

    niftiwrite(YC, outfile);
    compress_file = sprintf("gzip /home/zachkaras/fmri/motion_corrected/%s_mc.nii", name);

    system(compress_file);
    toc
%     break

end